In [1]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.proto import trainer_pb2, example_gen_pb2
import os
import tensorflow_data_validation as tfdv
import tensorflow as tf
from tensorflow_metadata.proto.v0 import anomalies_pb2
import pandas as pd
import tensorflow_transform as tft
import numpy as np
import tensorflow_model_analysis as tfma
from tfx.components import Pusher
from tfx.proto import pusher_pb2
from tfx.dsl.components.common.resolver import Resolver
from tfx.v1.dsl.experimental import LatestBlessedModelStrategy
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.types import Channel


2024-11-26 15:04:14.736296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 15:04:14.736435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 15:04:14.738315: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 15:04:14.748335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 15:04:17.058699: W tensorflow/compiler/tf2

In [2]:
pipeline_root = os.path.join(os.getcwd(), "rakhaeg-pipeline")
os.makedirs(pipeline_root, exist_ok=True)

In [3]:
context = InteractiveContext(pipeline_root=pipeline_root)


In [4]:

example_gen = CsvExampleGen(
    input_base='/home/rakha/proyek_5/data',
    output_config=example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
        ])
    ))
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [5]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [6]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [7]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [8]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.join(os.getcwd(), "transform_module.py")
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying random_forest_trainer.py -> build/lib
copying random_forest_model_adaptor.py -> build/lib
copying transform_module.py -> build/lib
copying app.py -> build/lib
copying calibrate_rf_model_adaptor.py -> build/lib


/home/rakha/proyek_5/my_venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmpwdd0uvdc
running install
running install_lib
copying build/lib/random_forest_trainer.py -> /tmp/tmpwdd0uvdc
copying build/lib/random_forest_model_adaptor.py -> /tmp/tmpwdd0uvdc
copying build/lib/transform_module.py -> /tmp/tmpwdd0uvdc
copying build/lib/app.py -> /tmp/tmpwdd0uvdc
copying build/lib/calibrate_rf_model_adaptor.py -> /tmp/tmpwdd0uvdc
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpwdd0uvdc/tfx_user_code_Transform-0.0+db0c52f50190d868ee0a12dbcb4a95a03d645297a5f9

2024-11-26 15:05:42.719523: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 15:05:42.784818: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 15:05:42.785027: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 15:05:42.792558: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 15:05:42.792757: I external/local_xla/xla/stream_executor

INFO:tensorflow:Assets written to: /home/rakha/proyek_5/rakhaeg-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/de4fe46debea40cab2beced2bbfac275/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [9]:
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri
print(f"Transformed Graph Path: {transform_graph_uri}")
transform_output = tft.TFTransformOutput(transform_graph_uri)
schema = transform_output.transformed_metadata.schema
print(schema)

Transformed Graph Path: /home/rakha/proyek_5/rakhaeg-pipeline/Transform/transform_graph/5
feature {
  name: "Connection Point_index"
  presence {
    min_fraction: 1
  }
  shape {
    dim {
      size: 1
    }
  }
  type: INT
  int_domain {
    min: -1
    max: 4
    is_categorical: true
  }
}
feature {
  name: "Delta Received Bytes"
  presence {
    min_fraction: 1
  }
  shape {
    dim {
      size: 1
    }
  }
  type: FLOAT
}
feature {
  name: "Delta Received Packets"
  presence {
    min_fraction: 1
  }
  shape {
    dim {
      size: 1
    }
  }
  type: FLOAT
}
feature {
  name: "Delta Sent Bytes"
  presence {
    min_fraction: 1
  }
  shape {
    dim {
      size: 1
    }
  }
  type: FLOAT
}
feature {
  name: "Delta Sent Packets"
  presence {
    min_fraction: 1
  }
  shape {
    dim {
      size: 1
    }
  }
  type: FLOAT
}
feature {
  name: "Received Bytes"
  presence {
    min_fraction: 1
  }
  shape {
    dim {
      size: 1
    }
  }
  type: FLOAT
}
feature {
  name: "Receiv

In [10]:

model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id('latest_blessed_model_resolver')

context.run(model_resolver)


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [11]:
resolved_model_artifacts = model_resolver.outputs['model'].get()

if resolved_model_artifacts:
    resolved_model_uri = resolved_model_artifacts[0].uri
    print(f"Model terbaru yang telah diberkati ditemukan di: {resolved_model_uri}")
    trainer_custom_config = {
        'baseline_model': resolved_model_uri+"/Format-Serving"
    }
else:
    print("Tidak ditemukan model yang telah diberkati sebelumnya.")
    trainer_custom_config = {}

Tidak ditemukan model yang telah diberkati sebelumnya.


In [12]:
trainer = Trainer(
    module_file='random_forest_trainer.py',
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50)
)
context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying random_forest_trainer.py -> build/lib
copying random_forest_model_adaptor.py -> build/lib
copying transform_module.py -> build/lib
copying app.py -> build/lib
copying calibrate_rf_model_adaptor.py -> build/lib


/home/rakha/proyek_5/my_venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmp5lc6s7tu
running install
running install_lib
copying build/lib/random_forest_trainer.py -> /tmp/tmp5lc6s7tu
copying build/lib/random_forest_model_adaptor.py -> /tmp/tmp5lc6s7tu
copying build/lib/transform_module.py -> /tmp/tmp5lc6s7tu
copying build/lib/app.py -> /tmp/tmp5lc6s7tu
copying build/lib/calibrate_rf_model_adaptor.py -> /tmp/tmp5lc6s7tu
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp5lc6s7tu/tfx_user_code_Trainer-0.0+db0c52f50190d868ee0a12dbcb4a95a03d645297a5f99ee7348d5c3e067243

Processing ./rakhaeg-pipeline/_wheels/tfx_user_code_Trainer-0.0+db0c52f50190d868ee0a12dbcb4a95a03d645297a5f99ee7348d5c3e06724388-py3-none-any.whl
Validasi sukses: Tidak ada kolom 'label' di train_x dan eval_x.
Evaluation Accuracy: 1.0000
Model disimpan sebagai pickle di /home/rakha/proyek_5/rakhaeg-pipeline/Trainer/model/7/Format-Serving/model.pkl
/home/rakha/proyek_5/rakhaeg-pipeline/Trainer/model/7/Format-Serving/model.pkl
INFO:tensorflow:Assets written to: /home/rakha/proyek_5/rakhaeg-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/rakha/proyek_5/rakhaeg-pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [19]:
import tensorflow_model_analysis as tfma

# Membuat konfigurasi evaluasi
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='label')  # Tentukan kolom label
    ],
    slicing_specs=[
        tfma.SlicingSpec(),  # Evaluasi seluruh dataset
        tfma.SlicingSpec(feature_keys=['Connection Point_index'])  # Slice berdasarkan fitur tertentu
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),  # Jumlah contoh
                tfma.MetricConfig(class_name='Accuracy',
                                  threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.8})
                    )),  # Akurasi
                tfma.MetricConfig(
                    class_name='AUC',  # Area Under Curve untuk evaluasi model
                    
                )
            ]
        )
    ]
)


In [20]:
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'], # Data evaluasi (biasanya keluaran dari komponen ExampleGen atau Transform)
    model=trainer.outputs['model'], # Model yang akan dievaluasi (keluaran dari komponen Trainer)
    eval_config=eval_config, # Konfigurasi evaluasi menggunakan tfma.EvalConfig
)
context.run(evaluator)  

/home/rakha/proyek_5/my_venv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/rakha/proyek_5/my_venv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/rakha/proyek_5/my_venv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/rakha/proyek_5/my_venv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/rakha/proyek_5/my_venv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  

ExecutionResult(
    component_id: Evaluator
    execution_id: 10
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [21]:
evaluation_artifacts = evaluator.outputs['evaluation'].get()

if evaluation_artifacts:
    print("Evaluator telah menghasilkan evaluasi.")
    for artifact in evaluation_artifacts:
        print(f"URI: {artifact.uri}")
else:
    print("Evaluator tidak menghasilkan artifacts evaluasi.")


Evaluator telah menghasilkan evaluasi.
URI: /home/rakha/proyek_5/rakhaeg-pipeline/Evaluator/evaluation/10


In [ ]:
evaluation_artifact = evaluator.outputs['evaluation'].get()[0]

evaluation_uri = evaluation_artifact.uri
eval_result = tfma.load_eval_result(evaluation_uri)

parsed_data = []

for slice_key, metrics in eval_result.slicing_metrics:
    # Determine slice name
    if slice_key:
        slice_name = f"{slice_key[0][1]}"
    else:
        slice_name = "All"

    # Extract metrics
    metrics_data = metrics['']['']['accuracy']['doubleValue']
    example_count = metrics['']['']['example_count']['doubleValue']
    auc = metrics['']['']['auc']['doubleValue']
    precision = metrics['']['']['precision']['doubleValue']
    recall = metrics['']['']['recall']['doubleValue']
    # Append parsed entry
    parsed_data.append({
        "Slice": slice_name,
        "Accuracy": metrics_data,
        "Example Count": example_count,
        "AUC": auc,
        
    })

# Create DataFrame
df = pd.DataFrame(parsed_data)
df

KeyError: 'precision'

In [17]:

pusher = Pusher(
    model=trainer.outputs['model'], 
    model_blessing=evaluator.outputs['blessing'],  
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join('model_serving')
        )
    )
)
context.run(pusher)


ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))